In [1]:
import pickle
import scipy.stats as stats
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from multiprocessing import Pool
from tqdm.notebook import tqdm
import cv2
import scipy.stats as stats

In [ ]:
yEval=pd.read_csv("data/test.csv")
tables = [pq.read_table('data/test_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
dfEval = pd.concat(tables)
dfEval = dfEval.set_index("image_id")

In [3]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [ ]:
size=(60,60)

def transformImg(img):
    img=255-img
    mu = cv2.moments(img, False)
    x, y= mu["m10"]/mu["m00"] , mu["m01"]/mu["m00"]
    M = np.float32([[1,0,(236/2)-x],[0,1,(137/2)-y]])
    img = cv2.warpAffine(img,M,(img.shape[1],img.shape[0]))
    img = cv2.resize(img, (size[0],size[1]))
    return img

imgs=dfEval.values.reshape(-1,137,236)
with Pool() as p:
    imap=p.imap(transformImg,imgs)
    imgs=list(tqdm(imap,total=imgs.shape[0]))

XEval = np.asarray(imgs).astype(np.float32)/255.0
XEval = XEval.reshape(-1,size[0],size[1],1)

In [ ]:
ans = pd.DataFrame(index=dfEval.index)

In [ ]:
for label in labels:
    y_preds = []
    for key, value in tqdm(db[label]):
        model = pickle.loads(value)
        try:
            y_pred = model.predict(XEval)
            y_pred = np.argmax(y_pred, axis=1)
            y_preds.append(y_pred)
        except:
            print(model)
    y_preds = np.asarray(y_preds)
    y_pred, _ = stats.mode(y_preds,axis=0)
    y_pred = y_pred.reshape(-1,)
    ans[label] = y_pred

In [ ]:
with open("submission.csv", "w") as fp:
    fp.write("row_id, target\n")
    for row_id,consonant,grapheme,vowel in zip(ans.index, ans["consonant_diacritic"],ans["grapheme_root"],ans["vowel_diacritic"]):
        fp.write("{0}_consonant_diacritic, {1}\n".format(row_id,consonant))
        fp.write("{0}_grapheme_root, {1}\n".format(row_id,grapheme))
        fp.write("{0}_vowel_diacritic, {1}\n".format(row_id,vowel))